In [2]:
# from numpy.linalg import inv
# from scipy import misc

# import numpy as np
# import matplotlib.pyplot as plt
# import math
# import pandas as pd
# import geopandas as gpd

# from shapely.geometry import Point

# import sklearn.preprocessing

# from sklearn.model_selection import train_test_split, GridSearchCV, GroupShuffleSplit, RepeatedStratifiedKFold
# from sklearn.metrics import precision_score, recall_score, roc_auc_score
from customGP import GaussianProcess


from dataloader import get_train_test


In [3]:
x_train, x_test, y_train, y_test = get_train_test(1000)

In [4]:
x_train

array([[ 43.030031 , -83.716307 ],
       [ 43.031349 , -83.73298  ],
       [ 43.036881 , -83.656913 ],
       ...,
       [ 43.071978 , -83.72185  ],
       [ 42.998548 , -83.660363 ],
       [ 43.03144  , -83.6397269]])

In [5]:
def get_custom_gp(X, y):
    GP = GaussianProcess(n_restarts=10, optimizer='L-BFGS-B')
    GP.fit(X, y.reshape(-1,1))
    return GP

In [ ]:
def plot_model(ax, model, X, y, title, decision_boundary = None, custom_gp = False):
    
    h = .02  # step size in the mesh

    x_min, x_max = X[:, 0].min() - .01, X[:, 0].max() + .01
    y_min, y_max = X[:, 1].min() - .01, X[:, 1].max() + .01
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # predict on the mesh
    x1v, x2v = np.meshgrid(np.linspace(x_min, x_max,100), np.linspace(y_min, y_max,100))
    xgrid = np.c_[x1v.ravel(), x2v.ravel()]
    
    if custom_gp:
        ygrid_prob, _ = model.predict(xgrid)
        ygrid_prob = ygrid_prob.reshape(x1v.shape)
        
        ax.set_title(title)
    else:
#         ygrid = model.predict(xgrid).reshape(x1v.shape)
        ygrid_prob = model.predict_proba(xgrid)[:,1].reshape(x1v.shape)
        
        ax.set_title(title + '\n' + f"log marginal likelihood={model.log_marginal_likelihood(model.kernel_.theta)}")
    
    ax.pcolormesh(x1v, x2v, ygrid_prob,alpha = 0.8, shading='auto')

    dangerous_indices = [i for i in range(len(y)) if y[i]==1]
    safe_indices = [i for i in range(len(y)) if y[i]==0]

    ax.scatter(X[dangerous_indices,0], X[dangerous_indices,1], marker= 'x', s=200)
    ax.scatter(X[safe_indices,0], X[safe_indices,1], marker= 'o', s=200, alpha=.1)
    
    # decision boundary
    if decision_boundary:
        cs = ax.contour(x1v, x2v, ygrid_prob, levels=[decision_boundary], colors='black', linestyles='dashed', linewidths=2)
        ax.clabel(cs, fontsize=20)

    ax.set_xlabel('Latitude')
    ax.set_ylabel('Longitude')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())